In [1]:
import numpy as np
import glob
import os
import cPickle as pickle
from collections import defaultdict
import sys

import shapefile as shp
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

import matplotlib.path as mplPath

import matplotlib.lines as mlines

import shutil
import seaborn as sns

from shutil import copyfile

%matplotlib inline

In [3]:
# eppy paramters
from eppy import modeleditor
from eppy.modeleditor import IDF
# set the idd file
iddfile = 'Energy+.idd'
IDF.setiddname(iddfile)

In [4]:
listFiles = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_control.idf')

In [5]:
# upgrade equip for these uses
possibles = [u'Mall', u'RetS', u'OffM', u'ReSd']

In [1]:
listPossibles = []
counter = 0
for counter,f in enumerate(listFiles):
    if counter%200==0:
        print counter
        sys.stdout.flush()
    idf1 = IDF(f)
    # find the uses
    people = idf1.idfobjects['PEOPLE']
    for j in range(len(people)):
        #print people[j].Number_of_People_Schedule_Name
        if any([substring in people[j].Number_of_People_Schedule_Name for substring in possibles]):
            listPossibles.append( f )
            break
# # save the list of offices
with open('../Intermediate_Results/listPossibles.pickle', 'wb') as outFile:
    pickle.dump(listPossibles, outFile, pickle.HIGHEST_PROTOCOL)

In [4]:
# with open('../Intermediate_Results/listPossibles.pickle', 'rb') as inFile:
#     listPossibles = pickle.load(inFile)

In [ ]:
print len(listPossibles)

In [ ]:
# equipment[j].Watts_per_Zone_Floor_Area
peakEqVals = {}
peakEqVals['OffM'] = []
peakEqVals['Mall'] = []
peakEqVals['RetS'] = []
peakEqVals['ReSd'] = []
for counter,f in enumerate(listPossibles):
    idf1 = IDF(f)
    equipment = idf1.idfobjects['ELECTRICEQUIPMENT']
    people = idf1.idfobjects['PEOPLE']
    # loop through the zone uses to find the office zones
    for j in range(len(people)):
        if 'OffM' in people[j].Number_of_People_Schedule_Name:
            peakEqVals['OffM'].append( equipment[j].Watts_per_Zone_Floor_Area)
        elif 'Mall' in people[j].Number_of_People_Schedule_Name:
            peakEqVals['Mall'].append( equipment[j].Watts_per_Zone_Floor_Area)
        if 'RetS' in people[j].Number_of_People_Schedule_Name:
            peakEqVals['RetS'].append( equipment[j].Watts_per_Zone_Floor_Area)
        elif 'ReSd' in people[j].Number_of_People_Schedule_Name:
            peakEqVals['ReSd'].append( equipment[j].Watts_per_Zone_Floor_Area)
            

In [ ]:
for p in possibles:
    print p, set(peakEqVals[p])

In [ ]:
# delete the unwanted old results
fileList = glob.glob('../Idf_generated_files/Commercial_equipment_efficiency/*.idf')
print len(fileList)
for file1 in fileList:
    os.remove(file1)

In [ ]:
# copy all office files to the Office_equipment_efficiency folder
for j in range(len(listPossibles)):
    idx = listPossibles[j].split('iency/')[1].split('_')[0]
    fileList = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*'+idx+'*')
    for f in fileList:
        if 'control' in f:
            newName = '../Idf_generated_files/Commercial_equipment_efficiency/'+idx+'_control_eqp.idf'
            copyfile(f, newName)
        elif 'experiment1' in f:
            newName = '../Idf_generated_files/Commercial_equipment_efficiency/'+idx+'_experiment1_eqp.idf'
            copyfile(f, newName)
        elif 'experiment2' in f:
            newName = '../Idf_generated_files/Commercial_equipment_efficiency/'+idx+'_experiment2_eqp.idf'
            copyfile(f, newName)            
        else:
            print 'we have a problem', f

In [ ]:
newValue = {}
for p in possibles:
    print set(peakEqVals[p])
    # we want a 10% increase in equipment efficiency
    #newValue = 10.76*0.9
    newValue[p] = peakEqVals[p][0]*0.9

In [ ]:
for p in newValue.keys():
    print 'For use', p, 'the new value is', newValue[p], 'whereas old value was', peakEqVals[p][0]

In [ ]:
listFiles = glob.glob('../Idf_generated_files/Commercial_equipment_efficiency/*')

In [ ]:
for counter, f in enumerate(listFiles):
    if counter%100==0:
        print counter
        sys.stdout.flush()
    idf1 = IDF(f)
    equipment = idf1.idfobjects['ELECTRICEQUIPMENT']
    people = idf1.idfobjects['PEOPLE']
    # loop through the zone uses to find the office zones
    for j in range(len(people)):           
        if 'OffM' in people[j].Number_of_People_Schedule_Name:
            equipment[j].Watts_per_Zone_Floor_Area = newValue['OffM']
        elif 'Mall' in people[j].Number_of_People_Schedule_Name:
            equipment[j].Watts_per_Zone_Floor_Area = newValue['Mall']
        if 'RetS' in people[j].Number_of_People_Schedule_Name:
            equipment[j].Watts_per_Zone_Floor_Area = newValue['RetS']
        elif 'ReSd' in people[j].Number_of_People_Schedule_Name:
            equipment[j].Watts_per_Zone_Floor_Area = newValue['ReSd']
    idf1.save()